In [36]:
from datasets import load_dataset, load_metric, Dataset, DatasetDict
from pathlib import Path
import pandas as pd
import numpy as np
from scipy.io import wavfile
import random
from IPython.display import display, HTML
import IPython.display as ipd
import re
import shutil
from sphfile import SPHFile
import json
from transformers import Wav2Vec2FeatureExtractor,  Wav2Vec2CTCTokenizer, Wav2Vec2Processor, Wav2Vec2ForCTC, TrainingArguments, Trainer
import torch
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

In [37]:
CWD = Path.cwd().parents[0]
DIR_DATA = CWD.joinpath('data', 'TIMIT')
DIR_TRAIN = DIR_DATA.joinpath('TRAIN')
DIR_TEST = DIR_DATA.joinpath('TEST')
FILE_METADATA_TRAIN = DIR_DATA.joinpath('train_metadata.csv')
FILE_METADATA_TEST = DIR_DATA.joinpath('test_metadata.csv')
GENERATE_DATA = False

In [38]:
num_threads = 28
torch.set_num_threads(num_threads)

In [39]:
def convert_strange_audio_format_to_wav(ind):
    """
    Convert the audios from some strange non-wav to wav format.
    """
    for file in ind.iterdir():
        if file.suffix == ".WAV" or file.suffix == ".wav":
            fname = file.parts[-1].split(".")[0]
            outf = ind.joinpath("{}_wav.wav".format(fname))
            sph = SPHFile(str(file))
            sph.write_wav(filename = str(outf))

In [40]:
def convert_files_to_wav():
    """
    Get through the train and test folder to convert the files.
    """
    # colect all folders into a list
    folders_to_process = []
    for folder in DIR_TEST.iterdir():
        if folder.is_dir():
            for next_folder in folder.iterdir():
                if next_folder.is_dir():
                    folders_to_process.append(next_folder)
                    
    
    for folder in DIR_TRAIN.iterdir():
        if folder.is_dir():
            for next_folder in folder.iterdir():
                if next_folder.is_dir():
                    folders_to_process.append(next_folder)

    # process the folders
    test = random.choice(folders_to_process)
    WAVS = [i for i in test.iterdir() if i.suffix == ".WAV"]
    assert len(WAVS) > 0 
    for folder in folders_to_process:
        convert_strange_audio_format_to_wav(ind=folder)

In [41]:
#convert_files_to_wav()

In [42]:
#convert_strange_audio_format_to_wav(ind=sample)

# Preprocess the Data

## Define utility functions for data preprocessing

In [43]:
def generate_data_file(indir=None, output_file=None):
    cnt = 1
    data = []
    for reg in indir.iterdir():
        dia_reg = reg.parts[-1]
        if reg.is_dir():
            for user in reg.iterdir():
                if user.is_dir():
                    for file in user.iterdir():
                        fname = file.parts[-1].split(".")[0]
                        if "wav" in fname:
                            cnt += 1
                            TXT_file = user.joinpath("{}.TXT".format(fname[:-4]))
                            try:
                                raw_txt = open(TXT_file, 'r').read()
                                txt = " ".join(raw_txt.split(" ")[2:])
                                txt = txt.replace("\n", "")
                            except:
                                print(raw_text)
                                pass
                            data_item = {'dialect_region': dia_reg, 'speaker_id': user.parts[-1],
                                        'aud_file': file, 'trasnc_file': TXT_file, 'text': txt}
                            data.append(data_item)
    df = pd.DataFrame(data)
    df['id'] = [i for i in range(1, len(df)+1)]
    df.to_csv(output_file, index=False)
    
    return df

In [44]:
def remove_special_charecters(txt):
    chars_to_ignore_regex = '[\,\),\?\.\!\-\;\:\"]'
    lower_txt = txt.lower()

    txt2 = re.sub(chars_to_ignore_regex, '', lower_txt)
    return txt2

In [45]:
def convert_files_to_wav(df):
    for idx, row in df.iterrows():
        try:
            curr_fpath = Path(row['aud_file'])
            fname = curr_fpath.parts[-1].split(".")[0]
            new_fpath = curr_fpath.joinpath(curr_fpath.parents[0], "{}_wav.wav".format(fname))
            sph = SPHFile(str(curr_fpath))
            sph.write_wav(filename = str(new_fpath))
            df.loc[idx, 'aud_file'] = str(new_fpath)
        except:
            print('Something went wrong')
    return df

In [46]:
def convert_weird_audio_format2wav():
    df = convert_files_to_wav(df=df_test)
    df.to_csv(FILE_METADATA_TEST, index=False)
    
    df = convert_files_to_wav(df=df_train)
    df.to_csv(FILE_METADATA_TRAIN, index=False)

In [47]:
def extract_all_chars(df, col):
    text_vals = list(df[col].values)
    all_text = " ".join(text_vals)
    vocab = list(set(all_text))
    return {"vocab": vocab, "all_text": all_text}

In [48]:
def convert_files_to_wav(ind):
    for idx, row in df.iterrows():
        try:
            curr_fpath = Path(row['aud_file'])
            fname = curr_fpath.parts[-1].split(".")[0]
            new_fpath = curr_fpath.joinpath(curr_fpath.parents[0], "{}_wav.wav".format(fname))
            sph = SPHFile(str(curr_fpath))
            sph.write_wav(filename = str(new_fpath))
            df.loc[idx, 'aud_file'] = str(new_fpath)
        except:
            print('Something went wrong')
    return df

## Process the data 

In [49]:
if GENERATE_DATA:
    df_test = generate_data_file(indir=DIR_TEST, output_file=FILE_METADATA_TEST)
    df_train = generate_data_file(indir=DIR_TRAIN, output_file=FILE_METADATA_TRAIN)
else:
    df_test = pd.read_csv(FILE_METADATA_TEST)
    df_train = pd.read_csv(FILE_METADATA_TRAIN)

In [50]:
def show_random_elements(df, num_examples=10):
    assert num_examples <= len(df), "Can't pick more elements than there are in the dataset."
    indices = df.index.values
    picks = random.choices(indices, k=num_examples)
    df = df.iloc[picks][['text']]
    display(HTML(df.to_html()))

In [51]:
show_random_elements(df=df_train, num_examples=10)

,text
2954,"When all else fails, use force."
115,Don't ask me to carry an oily rag like that.
1270,She had your dark suit in greasy wash water all year.
1031,To honor him is to honor ourselves.
3402,She had your dark suit in greasy wash water all year.
3126,"For a moment, boy and mount hung in midair."
4121,"Of course, dear, she said."
401,"She saw me and sat down beside me, three feet away."
1949,"His talk turns to what he calls the mess, or sometimes this buzzing confusion."
137,They used an aggressive policeman to flag thoughtless motorists.


In [52]:
df_train["text_no_quotes"] = df_train.text.apply(lambda x: remove_special_charecters(x))
df_test["text_no_quotes"] = df_test.text.apply(lambda x: remove_special_charecters(x))

# Generate Vocabulary

In [53]:
def generate_vocubalary(df_train, df_test, output_vocab_file):
    res_train = extract_all_chars(df_train, "text_no_quotes")
    res_test = extract_all_chars(df_test, "text_no_quotes")
    
    vocab_list = list(set(res_train["vocab"]) | set(res_test["vocab"]))
    
    vocab_dict = {v: k for k, v in enumerate(vocab_list)}
    vocab_dict["|"] = vocab_dict[" "]
    del vocab_dict[" "]
    
    vocab_dict["[UNK]"] = len(vocab_dict)
    vocab_dict["[PAD]"] = len(vocab_dict)
    

    with open(vocab_file, 'w') as file:
        json.dump(vocab_dict, file)

In [54]:
vocab_file = DIR_DATA.joinpath('vocab.json')
generate_vocubalary(df_train, df_test, output_vocab_file=vocab_file)

# Prepare Data for Model Training

In [55]:
tokenizer = Wav2Vec2CTCTokenizer(str(vocab_file), unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")
feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=False)
PROCESSOR = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [56]:
def prepare_dataset(row):
    sample_rate, audio = wavfile.read(row['aud_file'])
    
    if sample_rate != 16000:
        sample_rate = 16000

    # batched output is "un-batched" to ensure mapping is correct
    input_val = PROCESSOR(np.asarray(audio), sampling_rate=sample_rate).input_values[0]
    
    with PROCESSOR.as_target_processor():
        processed_txt = PROCESSOR(row["text"]).input_ids
    
    return input_val, processed_txt

In [57]:
df_train['input_values'] = df_train.apply(lambda x: prepare_dataset(x)[0], axis=1)
df_train['labels'] = df_train.apply(lambda x: prepare_dataset(x)[1], axis=1)

In [58]:
df_test['input_values'] = df_test.apply(lambda x: prepare_dataset(x)[0], axis=1)
df_test['labels'] = df_test.apply(lambda x: prepare_dataset(x)[1], axis=1)

In [59]:
cols_to_keep = ['input_values', 'labels']
dataset_train = Dataset.from_pandas(df_train[cols_to_keep])
dataset_test = Dataset.from_pandas(df_test[cols_to_keep])
ds = DatasetDict()

ds['train'] = dataset_train
ds['test'] = dataset_test 

# Train Model

In [60]:
@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [61]:
data_collator = DataCollatorCTCWithPadding(processor=PROCESSOR, padding=True)

In [62]:
#!pip install jiwer

In [63]:
try:
    wer_metric = load_metric("wer")
except ImportError:
    !pip install

In [ ]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = PROCESSOR.tokenizer.pad_token_id

    pred_str = PROCESSOR.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = PROCESSOR.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [65]:
model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-base", 
    ctc_loss_reduction="mean", 
    pad_token_id=PROCESSOR.tokenizer.pad_token_id,
)

loading configuration file https://huggingface.co/facebook/wav2vec2-base/resolve/main/config.json from cache at /home/ubuntu/.cache/huggingface/transformers/c7746642f045322fd01afa31271dd490e677ea11999e68660a92619ec7c892b4.ce1f96bfaf3d7475cb8187b9668c7f19437ade45fb9ceb78d2b06a2cec198015
/home/ubuntu/.local/lib/python3.6/site-packages/transformers/configuration_utils.py:359: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "
Model config Wav2Vec2Config {
  "activation_dropout": 0.0,
  "adapter_kernel_size": 3,
  "adapter_stride": 2,
  "add_adapter": false,
  "apply_spec_augment": true,
  "architectures": [
    "Wav2Vec2ForPreTraining"
  ],
  "attention_dr

In [66]:
model.freeze_feature_extractor()

/home/ubuntu/.local/lib/python3.6/site-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:1677: FutureWarning: The method `freeze_feature_extractor` is deprecated and will be removed in Transformers v5.Please use the equivalent `freeze_feature_encoder` method instead.
  FutureWarning,


In [74]:
model_output_dir = CWD.joinpath('models', 'eng_models')
epochs = 5

training_args = TrainingArguments(
  output_dir=model_output_dir,
  group_by_length=True,
  per_device_train_batch_size=16,
  evaluation_strategy="steps",
  num_train_epochs=epochs,
  gradient_checkpointing=True, 
  save_steps=500,
  eval_steps=500,
  logging_steps=500,
  learning_rate=1e-4,
  weight_decay=0.005,
  warmup_steps=1000,
  save_total_limit=2,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [75]:
trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=ds["train"],
    eval_dataset=ds["test"],
    tokenizer=PROCESSOR.feature_extractor,
)

In [76]:
from datetime import datetime

In [ ]:
start = datetime.now()
trainer.train()
end = datetime.now()
diff = (end - start).total_seconds()/60
print('Training {} epochs took {} minutes using {} cores'.format(epochs, int(diff), num_threads))

***** Running training *****
  Num examples = 4620
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 1445


Step,Training Loss,Validation Loss
